In [30]:
import pandas as pd
import numpy as np
import os
import glob
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, accuracy_score

In [31]:
output_csv = list()
for file_dir in glob.glob("./output_files/*.csv"):
    print(f"Loading... {file_dir}")
    df = pd.read_csv(file_dir)
    output_csv.append(df[["prediction"]])

Loading... ./output_files/base_lstm.csv
Loading... ./output_files/emsemble9.csv
Loading... ./output_files/catboost.csv
Loading... ./output_files/acc_submission.csv
Loading... ./output_files/best.csv


In [32]:
output_sum = list()
for df in output_csv:
    output_sum.append(list(df["prediction"].iloc[:744].values))

In [52]:
'''

'''

output_np = np.array(output_sum).T

row_len = len(output_np[0])
#print(row_len)

result = []

for row in output_np[:]:
    ans = np.where(row > 0.5,1,0) # voting
    if ans.sum() > row_len//2:
        value = np.average(row[np.where(ans == 1)]) # 1의 개수가 더 많으면
    elif ans.sum() < row_len//2:
        value = np.average(row[np.where(ans == 0)]) 
    else:
        value = np.average(row)
    
    result.append(value)
df['prediction'] = result
print ("Ensemble Done. Check ./ensemble_result/ensemble_submission.csv")

In [9]:
sum_res = np.sum(output_sum, axis=0) / len(output_csv)

df = pd.DataFrame()
df["prediction"] = sum_res
#df.to_csv("./ensemble_result/ensemble_submission.csv", index_label="id")
print ("Ensemble Done. Check ./ensemble_result/ensemble_submission.csv")

Ensemble Done. Check ./ensemble_result/ensemble_submission.csv


In [55]:
ground_truth = pd.read_csv("/opt/ml/input/Tester/ground_truth.csv")

auc = roc_auc_score(ground_truth['gt'], df['prediction'])
acc = accuracy_score(ground_truth['gt'], np.where(df['prediction'] >= 0.5, 1, 0))

print ("* Contains hidden testcase *")
print (f"auc : {auc}\nacc : {acc}")

* Contains hidden testcase *
auc : 0.8267044427901329
acc : 0.7473118279569892


In [11]:
df.to_csv("./ensemble_result/ensemble_submission.csv", index_label="id")